In [1]:
import sys
import numpy as np
import pandas as pd
import mml_lib.visualization as viz
sys.path.append('/home/jovyan/work/a091569/cython_home')

# cython utilities
from get_csv_rows import get_csv_rows
from csv_event_rows_to_dict import csv_event_rows_to_dict

ModuleNotFoundError: No module named 'csv_event_rows_to_dict'

In [82]:
!ls *.csv

product_attributes_csv_ur_develop_productattribute.csv
pros_cart_item_addn_csv_120_pros_cart_item_addition.csv
pros_cart_item_addn_csv_pros_cart_item_addition.csv
pros_cart_item_pur_csv_120_pros_cart_item_purchase.csv
pros_prod_vw_csv_120_pros_product_view.csv
results-20191022-152105.csv


In [83]:
view_rows = get_csv_rows('pros_prod_vw_csv_120_pros_product_view.csv',numcols=4)
print(len(view_rows))
view_rows[:3]

375938096


[['user_indiv_id', 'itemid', 'evnt_ts', 'sessionid'],
 ['161322486',
  '1806',
  '2019-06-26 17:46:22',
  '57133676298705600281061543070534137125503'],
 ['123152321',
  '64887',
  '2019-08-04 11:05:37',
  '756201711834091504050373584764359394489975620171183409150405037358476435939448']]

In [ ]:
view_cols = view_rows[0]
view_rows = pd.DataFrame(view_rows)
view_rows.columns = view_cols
view_rows.head()

In [1]:
view_rows[:3]

NameError: name 'view_rows' is not defined

In [65]:
#test = pd.read_csv('results-20191022-152105.csv')
test = pd.read_csv('pros_cart_item_addn_csv_pros_cart_item_addition.csv',sep='\t')
print(test.shape)
test.head()

(33046185, 4)


,user_indiv_id,itemid,evnt_ts,sessionid
0,23366872,237110,2019-08-25 12:10:03,9121353398468207352992220960641855277399121353...
1,167404204,106344,2019-10-17 03:31:25,8624261677556333870120113964743851738686242616...
2,1026487863,106344,2019-09-21 19:24:38,3262225133254954180461170530068461626474326222...
3,78588651,106344,2019-09-15 13:32:52,3342735317420968053691753015399636163211334273...
4,150977898,401011,2019-10-06 16:35:37,1696738881285883358215043740839664615810169673...


In [68]:
min(test['evnt_ts']), max(test['evnt_ts'])

('2019-08-20 00:00:00', '2019-10-18 23:59:59')

In [69]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33046185 entries, 0 to 33046184
Data columns (total 4 columns):
user_indiv_id    int64
itemid           int64
evnt_ts          object
sessionid        object
dtypes: int64(2), object(2)
memory usage: 1008.5+ MB


### To create a sequence of events we must:
* Group events by "vstr_id" or "user_indiv_id"
* Then sort by "evnt_ts"

In [70]:
sort1 = test.sort_values('evnt_ts')
sort1.head()

,user_indiv_id,itemid,evnt_ts,sessionid
13247240,112338792,9996280,2019-08-20 00:00:00,3297094789467973269691753025063303248617332970...
31075855,79323285,4648754,2019-08-20 00:00:01,8785495632687523678137183020366777391714187854...
18355531,120991693,1356233,2019-08-20 00:00:01,2099095209120684436434554810500908238517320990...
25503263,259798304,8315784,2019-08-20 00:00:01,9148496090015697266778932720021728587011914849...
9099149,44351390,9618868,2019-08-20 00:00:01,3328369169359421189691753027479225712573328369...


In [71]:
sort2 = test.copy()
sort2['evnt_ts'] = pd.to_datetime(sort2['evnt_ts'])
sort2 = sort2.sort_values('evnt_ts')
sort2.head()

,user_indiv_id,itemid,evnt_ts,sessionid
13247240,112338792,9996280,2019-08-20 00:00:00,3297094789467973269691753025063303248617332970...
31075855,79323285,4648754,2019-08-20 00:00:01,8785495632687523678137183020366777391714187854...
18355531,120991693,1356233,2019-08-20 00:00:01,2099095209120684436434554810500908238517320990...
25503263,259798304,8315784,2019-08-20 00:00:01,9148496090015697266778932720021728587011914849...
9099149,44351390,9618868,2019-08-20 00:00:01,3328369169359421189691753027479225712573328369...


In [73]:
#sort2.groupby(["vstr_id"]).apply(lambda x: x.sort_values(["evnt_ts"])).reset_index(drop=True)
grouptest2 = sort2.groupby(["user_indiv_id"]).apply(lambda x: x.sort_values(["evnt_ts"])).reset_index(drop=True)
grouptest2.head()

,user_indiv_id,itemid,evnt_ts,sessionid
0,-9223346256872843933,404141,2019-09-15 02:57:21,3542632177959033695706765902810329180713542632...
1,-9223346256872843933,585676,2019-09-18 22:14:21,3542632177959033695706765902810329180723542632...
2,-9223346256872843933,10216247,2019-09-27 22:37:25,3542632177959033695706765902810329180733542632...
3,-9223346256872843933,8336650,2019-09-27 22:39:03,3542632177959033695706765902810329180733542632...
4,-9223346256872843933,7975690,2019-09-27 22:40:52,3542632177959033695706765902810329180733542632...


In [75]:
user1 = grouptest2.query('user_indiv_id == -9223346256872843933')
user1

,user_indiv_id,itemid,evnt_ts,sessionid
0,-9223346256872843933,404141,2019-09-15 02:57:21,3542632177959033695706765902810329180713542632...
1,-9223346256872843933,585676,2019-09-18 22:14:21,3542632177959033695706765902810329180723542632...
2,-9223346256872843933,10216247,2019-09-27 22:37:25,3542632177959033695706765902810329180733542632...
3,-9223346256872843933,8336650,2019-09-27 22:39:03,3542632177959033695706765902810329180733542632...
4,-9223346256872843933,7975690,2019-09-27 22:40:52,3542632177959033695706765902810329180733542632...


In [78]:
viz.display_recom_panel([str(x) for x in user1['itemid']])

$60.0 (404141) 
 THOMSON PO... 
 
 $20.0 (585676) 
 GODINGER C... 
 
 $? (10216247) 
 ?... 
 
 $36.5 (8336650) 
 Karen Scot... 
 
 $64.5 (7975690) 
 Charter Cl...

In [80]:
grouptest2.shape

(33046185, 4)